In [1]:
# Install base packages
!pip install unsloth datasets transformers trl accelerate mlflow scikit-learn

# Uninstall and reinstall unsloth from GitHub for latest version
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# Install evaluation-specific packages
!pip install evaluate==0.4.2 rouge_score==0.1.2 bert-score==0.3.13

# Verify installed versions
!pip list | grep -E "unsloth|trl|transformers|torch|datasets|accelerate|mlflow|scikit-learn|evaluate|rouge_score|bert-score"

print("Packages installed and verified")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 76.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 63.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 110.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 45.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━

In [2]:
import os
import torch
import mlflow
from datasets import load_dataset, Dataset
from trl import SFTTrainer, SFTConfig
from transformers import DataCollatorForSeq2Seq
from unsloth import FastLanguageModel, is_bfloat16_supported
from accelerate import Accelerator
import shutil
import time
from evaluate import load
import numpy as np
from bert_score import score as bert_score

# GPU Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3  # GB
batch_size = 2  # T4-friendly
max_seq_length = 1024  # Reduced for speed
gradient_accumulation_steps = 2  # Reduced for faster iterations
dtype = torch.float16

print(f"Using GPU with {gpu_memory:.2f}GB memory")
print(f"Batch size: {batch_size}")
print(f"Pre-installed Torch version: {torch.__version__}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Using GPU with 14.74GB memory
Batch size: 2
Pre-installed Torch version: 2.6.0+cu124


In [5]:
# Load and preprocess dataset
dataset = load_dataset("roneneldan/TinyStories", split="train")
data = dataset.to_pandas().sample(n=2500, random_state=42)  # 2.5k rows for 1-hour tuning

# Create prompt-output pairs with explicit separation
data["instruction"] = data["text"].apply(lambda x: "Write a short story: " + x.split(".")[0] + ".")
data["response"] = data["text"]
train_data = Dataset.from_pandas(data[["instruction", "response"]])

# Format dataset with explicit tokenization for input and target
def format_example(example):
    input_text = example["instruction"]
    target_text = example["response"]
    inputs = tokenizer(input_text, truncation=True, max_length=max_seq_length // 2, padding="max_length", return_tensors="pt")
    targets = tokenizer(target_text, truncation=True, max_length=max_seq_length // 2, padding="max_length", return_tensors="pt")
    return {
        "input_ids": inputs["input_ids"][0].tolist(),
        "attention_mask": inputs["attention_mask"][0].tolist(),
        "labels": targets["input_ids"][0].tolist()
    }

train_dataset = train_data.map(format_example, remove_columns=["instruction", "response"])

print(f"Dataset size: {len(train_dataset)} rows")
print("Sample entry keys:", train_dataset[0].keys())

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Dataset size: 2500 rows
Sample entry keys: dict_keys(['__index_level_0__', 'input_ids', 'attention_mask', 'labels'])


In [4]:
# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True
)

# Apply LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # Reduced rank for speed
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42
)

print("Model and tokenizer loaded with LoRA applied")

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Model and tokenizer loaded with LoRA applied


In [6]:
# SFTConfig for training
sft_config = SFTConfig(
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    warmup_steps=5,
    max_steps=100,  # ~45 minutes on T4 x2 with 2.5k rows
    learning_rate=3e-4,  # Slightly higher for faster convergence
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=42,
    output_dir="/kaggle/working/outputs",
    report_to="none",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,  # Adjusted for efficiency
    packing=False  # No packing for simplicity
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    args=sft_config
)

print("Trainer initialized")

<ipython-input-6-b088e6b927ba>:22: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML (num_proc=2):   0%|          | 0/2500 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/2500 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/2500 [00:00<?, ? examples/s]

Trainer initialized


In [7]:
# MLflow setup
mlflow.set_tracking_uri("file:/kaggle/working/mlruns")
mlflow.set_experiment("Meta-Llama-3.1-8B-Instruct-bnb-4bit-1hr")

# Debug inputs (optional, comment out after verification)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True))
batch = next(iter(train_dataloader))
print("Sample batch keys:", batch.keys())
print("Sample input_ids shape:", batch["input_ids"].shape)
print("Sample labels shape:", batch["labels"].shape)

with mlflow.start_run():
    # Log hyperparameters
    mlflow.log_params({
        "model": "Meta-Llama-3.1-8B-Instruct-bnb-4bit",
        "learning_rate": 3e-4,
        "batch_size": batch_size,
        "max_steps": 100,
        "max_seq_length": max_seq_length,
        "gradient_accumulation_steps": gradient_accumulation_steps
    })

    # Train with timing
    print("Starting training...")
    start_time = time.time()
    trainer.train()
    training_time = time.time() - start_time
    mlflow.log_metric("training_time_seconds", training_time)

    # Save model
    model_save_path = "/kaggle/working/Llama3.1-8B-finetuned"
    accelerator = Accelerator()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)
    mlflow.log_artifacts(model_save_path, artifact_path="Llama3.1-8B-finetuned")

print(f"Training completed in {training_time:.2f} seconds")

2025/02/24 15:55:35 INFO mlflow.tracking.fluent: Experiment with name 'Meta-Llama-3.1-8B-Instruct-bnb-4bit-1hr' does not exist. Creating a new experiment.


Sample batch keys: dict_keys(['__index_level_0__', 'input_ids', 'attention_mask', 'labels'])
Sample input_ids shape: torch.Size([2, 512])
Sample labels shape: torch.Size([2, 512])
Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 2
\        /    Total batch size = 4 | Total steps = 100
 "-____-"     Number of trainable parameters = 20,971,520


Step,Training Loss


Training completed in 689.17 seconds


In [8]:
# Prepare for inference
FastLanguageModel.for_inference(model)

# Evaluation setup
print("Evaluating model...")
test_samples = train_data.select(range(50))  # Small test set for speed
prompts = [sample["instruction"] for sample in test_samples]
references = [sample["response"] for sample in test_samples]
predictions = [
    tokenizer.decode(model.generate(**tokenizer(prompt, return_tensors="pt").to(device), max_new_tokens=50, do_sample=True, temperature=0.7)[0], skip_special_tokens=True)
    for prompt in prompts
]

print("Predictions generated")

Evaluating model...
Predictions generated


In [9]:
# Zip outputs for download
shutil.make_archive("/kaggle/working/llama3.1-8B-finetuned", "zip", model_save_path)
shutil.make_archive("/kaggle/working/mlruns", "zip", "/kaggle/working/mlruns")
shutil.make_archive("/kaggle/working/outputs", "zip", "/kaggle/working/outputs")

print("Outputs zipped and ready for download")

Outputs zipped and ready for download


In [10]:
# 1. Perplexity
def compute_perplexity(model, tokenizer, texts):
    total_loss = 0
    total_tokens = 0
    model.eval()
    with torch.no_grad():
        for text in texts:
            inputs = tokenizer(text, return_tensors="pt").to(device)
            outputs = model(**inputs, labels=inputs["input_ids"])
            total_loss += outputs.loss.item() * inputs["input_ids"].size(1)
            total_tokens += inputs["input_ids"].size(1)
    perplexity = np.exp(total_loss / total_tokens)
    return perplexity

perplexity = compute_perplexity(model, tokenizer, references)
mlflow.log_metric("perplexity", perplexity)

# 2. ROUGE-L
from evaluate import load
rouge = load("rouge")  # Removed download_config to avoid AttributeError
rouge_scores = rouge.compute(predictions=predictions, references=references)
rouge_l = rouge_scores["rougeL"]
mlflow.log_metric("rouge_l", rouge_l)

# 3. BERTScore
from bert_score import score as bert_score
P, R, F1 = bert_score(predictions, references, lang="en", verbose=False)
bertscore_f1 = F1.mean().item()
mlflow.log_metric("bertscore_f1", bertscore_f1)

# 4. Self-BLEU
bleu = load("bleu")
self_bleu_scores = []
for i, pred in enumerate(predictions):
    refs = predictions[:i] + predictions[i+1:]
    if refs:  # Ensure refs is not empty
        score = bleu.compute(predictions=[pred], references=[refs])["bleu"]
        self_bleu_scores.append(score)
self_bleu = np.mean(self_bleu_scores) if self_bleu_scores else 0
mlflow.log_metric("self_bleu", self_bleu)

# 5. Distinct-N (Distinct-2)
def compute_distinct_n(texts, n=2):
    all_ngrams = set()
    total_ngrams = 0
    for text in texts:
        tokens = text.split()
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            all_ngrams.add(ngram)
            total_ngrams += 1
    return len(all_ngrams) / total_ngrams if total_ngrams > 0 else 0

distinct_2 = compute_distinct_n(predictions, n=2)
mlflow.log_metric("distinct_2", distinct_2)

# Print metrics
print(f"Perplexity: {perplexity:.2f}")
print(f"ROUGE-L: {rouge_l:.4f}")
print(f"BERTScore F1: {bertscore_f1:.4f}")
print(f"Self-BLEU: {self_bleu:.4f}")
print(f"Distinct-2: {distinct_2:.4f}")

AttributeError: 'DownloadConfig' object has no attribute 'use_auth_token'

In [11]:
# 1. Perplexity
def compute_perplexity(model, tokenizer, texts):
    total_loss = 0
    total_tokens = 0
    model.eval()
    with torch.no_grad():
        for text in texts:
            inputs = tokenizer(text, return_tensors="pt").to(device)
            outputs = model(**inputs, labels=inputs["input_ids"])
            total_loss += outputs.loss.item() * inputs["input_ids"].size(1)
            total_tokens += inputs["input_ids"].size(1)
    perplexity = np.exp(total_loss / total_tokens)
    return perplexity

perplexity = compute_perplexity(model, tokenizer, references)
mlflow.log_metric("perplexity", perplexity)

print(f"Perplexity: {perplexity:.2f}")

Perplexity: 7.78


In [12]:
# 2. BERTScore
from bert_score import score as bert_score

P, R, F1 = bert_score(predictions, references, lang="en", verbose=False)
bertscore_f1 = F1.mean().item()
mlflow.log_metric("bertscore_f1", bertscore_f1)

print(f"BERTScore F1: {bertscore_f1:.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8138


In [13]:
# 3. Self-BLEU
from evaluate import load

bleu = load("bleu")
self_bleu_scores = []
for i, pred in enumerate(predictions):
    refs = predictions[:i] + predictions[i+1:]
    if refs:
        score = bleu.compute(predictions=[pred], references=[refs])["bleu"]
        self_bleu_scores.append(score)
self_bleu = np.mean(self_bleu_scores) if self_bleu_scores else 0
mlflow.log_metric("self_bleu", self_bleu)

print(f"Self-BLEU: {self_bleu:.4f}")

AttributeError: 'DownloadConfig' object has no attribute 'use_auth_token'

In [14]:
# 4. Distinct-N (Distinct-2)
def compute_distinct_n(texts, n=2):
    all_ngrams = set()
    total_ngrams = 0
    for text in texts:
        tokens = text.split()
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            all_ngrams.add(ngram)
            total_ngrams += 1
    return len(all_ngrams) / total_ngrams if total_ngrams > 0 else 0

distinct_2 = compute_distinct_n(predictions, n=2)
mlflow.log_metric("distinct_2", distinct_2)

print(f"Distinct-2: {distinct_2:.4f}")

Distinct-2: 0.4894


In [15]:
# 5. ROUGE-L (Isolated due to persistent error)
try:
    from evaluate import load
    rouge = load("rouge")
    rouge_scores = rouge.compute(predictions=predictions, references=references)
    rouge_l = rouge_scores["rougeL"]
    mlflow.log_metric("rouge_l", rouge_l)
    print(f"ROUGE-L: {rouge_l:.4f}")
except AttributeError as e:
    print(f"ROUGE-L failed with error: {e}")
    print("Skipping ROUGE-L due to compatibility issue with evaluate and datasets versions")

ROUGE-L failed with error: 'DownloadConfig' object has no attribute 'use_auth_token'
Skipping ROUGE-L due to compatibility issue with evaluate and datasets versions
